In [1]:
import numpy as np
import cv2


In [13]:
#读入数据
points = []
with open('points.txt', 'r') as f:
    num_points = int(f.readline().strip())
    for _ in range(num_points):
        point = list(map(float, f.readline().strip().split()))
        points.append(point)

points = np.array(points)

K = np.array([[400, 0, 190],
              [0, 400, 160],
              [0, 0, 1]])

cam_position = np.array([2, 2, 2])
quaternion = np.array([-0.5, 0.5, 0.5, -0.5])


In [14]:
#四元数转矩阵
def quaternion_to_mat(Q):
    w, x, y, z = Q
    return np.array([[1 - 2*(y**2 + z**2), 2*(x*y - z*w), 2*(x*z + y*w)],
                    [2*(x*y + z*w), 1 - 2*(x**2 + z**2), 2*(y*z - x*w)],
                    [2*(x*z - y*w), 2*(y*z + x*w), 1 - 2*(x**2 + y**2)]])

R = quaternion_to_mat(quaternion)


In [22]:
res_points = []

for point in points:
    cam_point = R.T @ (point - cam_position)

    if cam_point[2] > 0:  # 点在相机前面，变换后加入图上点集
        proj_point = K @ cam_point
        proj_point /= proj_point[2] 
        res_points.append((int(proj_point[0]), int(proj_point[1])))

print(len(res_points))

# 创建图像，绘制点
image = np.zeros((1000, 1200, 3), dtype=np.uint8)
for pt in res_points:
    cv2.circle(image, pt, 3, (0, 255, 0), -1)

cv2.imwrite("./output.jpg", image)

cv2.imshow("Projected Points", image)
cv2.waitKey(0)
cv2.destroyAllWindows()



2048
